In [13]:
import os
import polars as pl

storage_options = {
    "aws_endpoint":  'https://'+'minio-simple.lab.groupe-genes.fr',
    "aws_access_key_id": os.environ["AWS_ACCESS_KEY_ID"],
    "aws_secret_access_key": os.environ["AWS_SECRET_ACCESS_KEY"],
    "aws_region": os.environ["AWS_DEFAULT_REGION"],
    "aws_token": os.environ["AWS_SESSION_TOKEN"]
  }
username = os.environ["VAULT_TOP_DIR"] # un peu bancal pour avoir le username

df = pl.read_parquet(source = f"s3://{username}/diffusion/INSEE/prenoms-nat2022.parquet", storage_options=storage_options)
print(df)

shape: (703_007, 4)
┌──────┬────────────────┬────────┬────────┐
│ sexe ┆ preusuel       ┆ annais ┆ nombre │
│ ---  ┆ ---            ┆ ---    ┆ ---    │
│ i64  ┆ str            ┆ str    ┆ i64    │
╞══════╪════════════════╪════════╪════════╡
│ 1    ┆ _PRENOMS_RARES ┆ 1900   ┆ 1249   │
│ 1    ┆ _PRENOMS_RARES ┆ 1901   ┆ 1342   │
│ 1    ┆ _PRENOMS_RARES ┆ 1902   ┆ 1330   │
│ 1    ┆ _PRENOMS_RARES ┆ 1903   ┆ 1286   │
│ 1    ┆ _PRENOMS_RARES ┆ 1904   ┆ 1430   │
│ …    ┆ …              ┆ …      ┆ …      │
│ 2    ┆ ZYNEB          ┆ 2019   ┆ 7      │
│ 2    ┆ ZYNEB          ┆ 2020   ┆ 8      │
│ 2    ┆ ZYNEB          ┆ 2021   ┆ 6      │
│ 2    ┆ ZYNEB          ┆ 2022   ┆ 4      │
│ 2    ┆ ZYNEB          ┆ XXXX   ┆ 19     │
└──────┴────────────────┴────────┴────────┘


In [21]:
top10f2021 = df\
    .filter((pl.col("sexe") == 2) & 
            (pl.col("annais") == "2021") & 
            (pl.col("preusuel") != "_PRENOMS_RARES"))

In [24]:
top10f2021 = df\
    .filter((pl.col("sexe") == 2) & 
            (pl.col("annais") == "2021") & 
            (pl.col("preusuel") != "_PRENOMS_RARES"))\
    .group_by("preusuel")\
    .agg(pl.col("nombre").sum().alias("nombre_total"))\
    .sort("nombre_total", descending=True)\
    .limit(10)

print(top10f2021)




shape: (10, 2)
┌──────────┬──────────────┐
│ preusuel ┆ nombre_total │
│ ---      ┆ ---          │
│ str      ┆ i64          │
╞══════════╪══════════════╡
│ JADE     ┆ 3800         │
│ LOUISE   ┆ 3768         │
│ EMMA     ┆ 3201         │
│ AMBRE    ┆ 3017         │
│ ALICE    ┆ 2767         │
│ ROSE     ┆ 2702         │
│ ANNA     ┆ 2514         │
│ ALBA     ┆ 2505         │
│ ROMY     ┆ 2447         │
│ MIA      ┆ 2431         │
└──────────┴──────────────┘


In [30]:
import s3fs

fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio-simple.lab.groupe-genes.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

destination = f"s3://{username}/diffusion/INSEE/output.csv"

with fs.open(destination, mode='wb') as f:
    top10f2021.write_csv(f)

/tmp/ipykernel_1213/274316182.py:13: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  top10f2021.write_csv(f)
